# freehand_polygon_counter — count pixels inside a freehand polygon (full-res)

**Purpose:** Open a full-resolution image, let you draw a freehand polygon, and report the pixel count **in original resolution** (no post-resize blur in the mask).

**Controls**
- **Hold LEFT mouse** — draw the polygon freehand  
- **Release + RIGHT click** — finish polygon & count pixels  
- **r** — reset view  
- **q** or **Esc** — quit

**Tip:** The image is displayed downscaled to fit your screen, but the mask and pixel count are computed at the **original** resolution.



In [ ]:
from pathlib import Path
import sys

import cv2
import numpy as np

## Determine display scale (without changing the counting resolution)

We detect your screen size (via `tkinter` if available) and compute a **display scale ≤ 1** so the image fits on screen.  
The polygon is always recorded in **original pixel coordinates**, so counting is done on the full-resolution mask.


In [ ]:
try:                                               # works on Windows / macOS / X11
    import tkinter as tk

    def _screen_size() -> tuple[int, int]:
        root = tk.Tk()
        root.withdraw()
        w, h = root.winfo_screenwidth(), root.winfo_screenheight()
        root.destroy()
        return w, h
except Exception:                                 
    def _screen_size() -> tuple[int, int]:
        return 1920, 1080                         # safe default


## FreehandCounter — draw, close, count

- Maintains `self.base` (original image) and `self.mask` (original-size mask).
- Displays a scaled copy (`self.show`) using `INTER_AREA` for clean downscaling.
- Mouse callback:
  - left-drag appends points (stored in **original** coordinates)
  - right-click closes and fills polygon → counts pixels in the full-res mask
- `r` resets the display; `q`/`Esc` quits.


In [ ]:
class FreehandCounter:
    def __init__(self, img: np.ndarray):
        self.base = img                             # full-resolution reference
        screen_w, screen_h = _screen_size()

        h, w = img.shape[:2]
        self.scale = min(1.0, screen_w / w, screen_h / h)  # ≤ 1 → shrink only
        self.disp_size = (int(w * self.scale), int(h * self.scale))

        # working copy that the user actually sees
        self.show = cv2.resize(img, self.disp_size, interpolation=cv2.INTER_AREA)
        self.mask  = np.zeros(img.shape[:2], np.uint8)     # same size as base
        self.points: list[tuple[int, int]] = []            # polygon vertices (orig px)
        self.drawing = False

        cv2.namedWindow("Freehand Counter", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("Freehand Counter", self.show)
        cv2.setMouseCallback("Freehand Counter", self._mouse_cb)

    # coordinate conversion 
    def _to_orig(self, x: int, y: int) -> tuple[int, int]:
        """display->original coordinate system"""
        return int(round(x / self.scale)), int(round(y / self.scale))

    def _to_disp(self, x: int, y: int) -> tuple[int, int]:
        """original->display coordinate system"""
        return int(round(x * self.scale)), int(round(y * self.scale))

    # mouse callback
    def _mouse_cb(self, event, x, y, flags, _):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.points = [self._to_orig(x, y)]
        elif event == cv2.EVENT_MOUSEMOVE and self.drawing:
            ox, oy = self._to_orig(x, y)
            self.points.append((ox, oy))
            cv2.line(
                self.show,
                self._to_disp(*self.points[-2]),
                (x, y),                       # current point already in display px
                (0, 255, 255), 1
            )
        elif event == cv2.EVENT_LBUTTONUP and self.drawing:
            self.drawing = False
        elif event == cv2.EVENT_RBUTTONDOWN and self.points:
            self._finish_polygon()

    # close & count 
    def _finish_polygon(self):
        if len(self.points) < 3:
            print("Need a closed shape. Hold left button to draw.")
            self._reset()
            return

        pts = np.array(self.points, dtype=np.int32)
        self.mask[:] = 0
        cv2.fillPoly(self.mask, [pts], 255)
        inside = int((self.mask > 0).sum())
        print(f"Pixels inside polygon (full-res): {inside}")

        overlay = self.base.copy()
        overlay[self.mask > 0] = (0, 0, 255)                  # paint selection red
        blended = cv2.addWeighted(self.base, 0.6, overlay, 0.4, 0)

        self.show = cv2.resize(blended, self.disp_size, interpolation=cv2.INTER_AREA)
        self.points.clear()

    # reset 
    def _reset(self):
        self.show = cv2.resize(self.base, self.disp_size, interpolation=cv2.INTER_AREA)
        self.points.clear()
        self.drawing = False

    # main loop 
    def run(self):
        print("Hold left-click to draw, right-click to finish, r to reset, q/Esc to quit")
        while True:
            cv2.imshow("Freehand Counter", self.show)
            k = cv2.waitKey(20) & 0xFF
            if k in (ord("q"), 27):
                break
            elif k == ord("r"):
                self._reset()
        cv2.destroyAllWindows()

## Quick start (Notebook)

Specify an image path and run the cell to launch the interactive window.  
(If your environment blocks GUI windows, run this as a CLI script instead.)


In [ ]:
image_path = "path/to/your/image.jpg" 

img = cv2.imread(str(image_path))
if img is None:
    raise FileNotFoundError(f"Cannot open: {image_path}")

FreehandCounter(img).run()


## CLI entry point (optional)

If you prefer to run from the command line, keep the following cell.  
In a script file, run: `python freehand_polygon_counter.py path/to/image.jpg`


In [ ]:
def main(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        print("Cannot open:", path)
        sys.exit(1)
    FreehandCounter(img).run()


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python freehand_polygon_counter.py image.jpg")
        sys.exit(1)
    main(Path(sys.argv[1]))
